In [64]:
pip install boto3

                                              0.0/12.2 MB ? eta -:--:--
     -                                        0.4/12.2 MB 12.2 MB/s eta 0:00:01
     -----                                    1.6/12.2 MB 20.5 MB/s eta 0:00:01
     ----------                               3.1/12.2 MB 24.9 MB/s eta 0:00:01
     --------------                           4.3/12.2 MB 24.9 MB/s eta 0:00:01
     -------------------                      5.8/12.2 MB 26.5 MB/s eta 0:00:01
     --------------------                     6.2/12.2 MB 23.3 MB/s eta 0:00:01
     ---------------------------              8.3/12.2 MB 26.7 MB/s eta 0:00:01
     --------------------------------         9.8/12.2 MB 27.3 MB/s eta 0:00:01
     --------------------------------------  12.1/12.2 MB 32.8 MB/s eta 0:00:01
     --------------------------------------  12.2/12.2 MB 32.8 MB/s eta 0:00:01
     --------------------------------------- 12.2/12.2 MB 27.3 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found exist

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.34.90 which is incompatible.
awscli 1.32.59 requires botocore==1.34.59, but you have botocore 1.34.90 which is incompatible.


In [66]:
from kafka import KafkaConsumer
import json
import boto3

# Initialize the S3 filesystem
s3_client = boto3.client('s3')

In [69]:
# Initialize Kafka consumer to subscribe to multiple topics
consumer = KafkaConsumer(
    'healthcare-provider-records',
    'lifestyle-and-diet-surveys',
    'nutritional-data-systems',
    bootstrap_servers=['54.208.97.228:9092'],  # Add your IP here
    group_id='group1',
    value_deserializer=lambda x: json.loads(x.decode('utf-8')))

In [73]:
bucket_name = 'mcgill-mma-enterpriseanalytics-project-yash'
# Define a mapping from topic names to folder names
folder_map = {
    'healthcare-provider-records': 'healthcare-provider-records',
    'lifestyle-and-diet-surveys': 'lifestyle-and-diet-surveys',
    'nutritional-data-systems': 'nutritional-data-systems'
}

# Process messages
for count, message in enumerate(consumer):
    if message.value is None:
        continue

    # Determine the folder based on the message's topic
    folder_name = folder_map.get(message.topic)

    # Serialize the message value to JSON
    message_json = json.dumps(message.value)
    
    # Generate the S3 key (path) for the JSON file
    s3_key = f"{folder_name}/record_{count}.json"

    try:
        # Upload the JSON data to S3
        s3_client.put_object(
            Bucket=bucket_name,
            Key=s3_key,
            Body=message_json,
            ContentType='application/json'
        )
        print(f"Successfully uploaded record_{count}.json to {bucket_name}/{s3_key}")
        
        # Commit the message offset after processing
        consumer.commit()
    except Exception as e:
        print(f"Failed to upload to S3: {e}")

Successfully uploaded record_0.json to mcgill-mma-enterpriseanalytics-project-yash/healthcare-provider-records/record_0.json
Successfully uploaded record_1.json to mcgill-mma-enterpriseanalytics-project-yash/healthcare-provider-records/record_1.json
Successfully uploaded record_2.json to mcgill-mma-enterpriseanalytics-project-yash/healthcare-provider-records/record_2.json
Successfully uploaded record_3.json to mcgill-mma-enterpriseanalytics-project-yash/healthcare-provider-records/record_3.json
Successfully uploaded record_4.json to mcgill-mma-enterpriseanalytics-project-yash/healthcare-provider-records/record_4.json
Successfully uploaded record_5.json to mcgill-mma-enterpriseanalytics-project-yash/healthcare-provider-records/record_5.json
Successfully uploaded record_6.json to mcgill-mma-enterpriseanalytics-project-yash/healthcare-provider-records/record_6.json
Successfully uploaded record_7.json to mcgill-mma-enterpriseanalytics-project-yash/healthcare-provider-records/record_7.json
